# Imports, install and mount

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
my_local_drive='/content/gdrive/MyDrive/SL2024/datas'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive
%pwd
print("DRIVE MOUNTED")

Mounted at /content/gdrive
/content/gdrive/MyDrive/SL2024/datas
DRIVE MOUNTED


<!--  -->

In [8]:
# ! pip install cuda
# ! pip install torch_geometric
# ! pip install nxontology
# ! pip install tensordict
# ! pip install pandas
# ! pip install tensorflow
# ! pip install scipy
# ! pip install matplotlib

import random
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import os.path as osp
from torch_geometric.nn import ComplEx
from torch_geometric.data import Data
import pandas as pd
from tqdm import tqdm
from torch_geometric.loader import DataLoader

from torch.nn.functional import normalize
from tensordict.tensordict import TensorDict
import torch.nn.functional as F

from nxontology.imports import from_file


# Settings


In [12]:
# ComplEx embeddings :

hidden_channels = 10
batch_size = 4096
epochs = 1000
file_path = '/content/gdrive/MyDrive/SL2024/datas/genes_to_phenotypes_iric.tsv'
# file_path = r"C:\Users\Admin\Desktop\Stage_LIRMM_2024\ESL2024\DATAS\little_Os_GO_iric.tsv"
# file_path = "/home/elliot/Documents/ESL2024/data/little_iric.tsv"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu' # Tip : Use cpu for debugging
print(device)


# Losses tests :

n_negative_triplets_per_positive = 4 # Number of false negatives triples per positive triple in the dataset for losses tests.
url = "https://purl.obolibrary.org/obo/go/go-basic.json.gz" # Where should I look for GO ?

cpu


# Preparing ComplEx embeddings

## TSV to pyg.Data

We want :
- x (tensorised and processed node attributes) (1 for now)
- edge_index (a tensor of shape (num_edges, 2) indicates the source node index and the destination node index)
- edge_attr (a tensor that indicate the type of each edge)
- y (edge labels - optional, can be defined as node labels if needed)

## Reading and mapping graph

What we want : Create a Data object with all the properties I want to use later

	- x (tensorised and processed node attributes) (Not for now)
	- edge_index (a tensor of shape (num_edges, 2) indicates the source node index and the destination node index)
	- y (desired edge labels - optional, can be defined as node labels if needed) (Not for now)
	- any other things you want to use later

In [13]:
iric = pd.read_csv(file_path, delimiter='\t', names = ['subject', 'predicate','object'])
display(iric)

# Mapping entities to ids

entity_set = set(iric['object']).union(set(iric['subject']))
entity_to_mapping = {entity: int(i) for i, entity in enumerate(entity_set)}
relation_set = set(iric['predicate'])
relation_to_mapping = {relation: int(i) for i, relation in enumerate(relation_set)}

mapped_iric = iric.copy()
mapped_iric['object'] = mapped_iric['object'].apply(lambda x: entity_to_mapping[x])
mapped_iric['subject'] = mapped_iric['subject'].apply(lambda x: entity_to_mapping[x])
mapped_iric['predicate'] = mapped_iric['predicate'].apply(lambda x: relation_to_mapping[x])

display(mapped_iric)

print('OK')
print("Minima in mappings :")
print('subject :', min(mapped_iric['subject']))
print('predicate :',min(mapped_iric['predicate']))
print('object :',min(mapped_iric['object']))

print(entity_to_mapping)
print(relation_to_mapping)

mapping_to_entity = {v: k for k, v in entity_to_mapping.items()}
mapping_to_relation = {v: k for k, v in relation_to_mapping.items()}

print("Mapping to entity dict :",mapping_to_entity)
print("Mapping to relation dict",mapping_to_relation)

,subject,predicate,object
0,OsNippo01g010050,gene ontology,GO:0031267
1,OsNippo01g010050,gene ontology,GO:0006886
2,OsNippo01g010050,gene ontology,GO:0005622
3,OsNippo01g010050,gene ontology,GO:0005623
4,OsNippo01g010050,gene ontology,GO:0090630
...,...,...,...
169243,OsNippo12g248550,gene ontology,GO:0009409
169244,OsNippo12g248550,gene ontology,GO:0001666
169245,OsNippo12g250550,gene ontology,GO:0008270
169246,OsNippo12g255100,gene ontology,GO:0005576


,subject,predicate,object
0,19922,0,12951
1,19922,0,2959
2,19922,0,17945
3,19922,0,10460
4,19922,0,14773
...,...,...,...
169243,3828,0,30272
169244,3828,0,5346
169245,6117,0,18838
169246,4919,0,16619


OK
Minima in mappings :
subject : 0
predicate : 0
object : 4
{'OsNippo07g060100': 0, 'OsNippo07g129200': 1, 'OsNippo08g019550': 2, 'OsNippo07g054100': 3, 'GO:0008187': 4, 'OsNippo12g078200': 5, 'OsNippo01g430900': 6, 'OsNippo07g134000': 7, 'OsNippo02g322900': 8, 'GO:0010857': 9, 'OsNippo07g228550': 10, 'OsNippo04g299650': 11, 'OsNippo08g201400': 12, 'OsNippo01g119600': 13, 'OsNippo02g274950': 14, 'OsNippo09g107700': 15, 'OsNippo06g149550': 16, 'OsNippo07g066750': 17, 'GO:0055121': 18, 'OsNippo10g056600': 19, 'GO:0009991': 20, 'GO:0001188': 21, 'OsNippo06g098200': 22, 'GO:0044237': 23, 'OsNippo03g105000': 24, 'OsNippo01g055950': 25, 'OsNippo03g115700': 26, 'OsNippo10g025050': 27, 'OsNippo05g258550': 28, 'OsNippo08g204100': 29, 'OsNippo11g138850': 30, 'OsNippo12g226400': 31, 'OsNippo04g235000': 32, 'OsNippo08g071700': 33, 'GO:0003922': 34, 'OsNippo11g275600': 35, 'OsNippo12g211600': 36, 'OsNippo03g249550': 37, 'OsNippo09g179750': 38, 'OsNippo04g289300': 39, 'OsNippo10g038400': 40, 'OsNip

## Building init vars for Data :

In [14]:
# # Initial nodes states :
# x = torch.ones(len(entity_set), 1)  # Chaque nœud a 1 pour état initial
# print('X : \n',x)

# Edges index
heads = list(mapped_iric['subject'])
tails = list(mapped_iric['object'])
edge_index = torch.tensor([heads,tails], dtype=torch.long)
print('\nEDGE INDEX : \n',edge_index)

# edges states
edge_attributes = torch.tensor(mapped_iric['predicate'])
print('\nEDGES ATTRIBUTES : \n',edge_attributes)

iric_pyg = Data(
                # x = x,
                num_nodes = len(entity_set),
                edge_index = edge_index,
                edge_attr = edge_attributes)
print('\nDATASET :\n',iric_pyg)


print("\nDataset looks valid ? \n",iric_pyg.validate(raise_on_error=True))


EDGE INDEX : 
 tensor([[19922, 19922, 19922,  ...,  6117,  4919,  4919],
        [12951,  2959, 17945,  ..., 18838, 16619,  3535]])

EDGES ATTRIBUTES : 
 tensor([0, 0, 0,  ..., 0, 0, 0])

DATASET :
 Data(edge_index=[2, 169248], edge_attr=[169248], num_nodes=30396)

Dataset looks valid ? 
 True


## Setting up datas and model


## Splitting dataset

In [15]:
from torch_geometric.transforms import RandomLinkSplit

transform = RandomLinkSplit(
                            num_val = 0.1,
                            num_test = 0.1,
                            is_undirected=False,
                            add_negative_train_samples=False,
                            )

train_data, val_data, test_data = transform(iric_pyg)

print(f"Hole Dataset :\n {iric_pyg}\n\nTrain:\n{train_data}\n\nTest :\n{test_data}\n\nValidation :\n{val_data}")
# Il ne faut pas regarder num_edges parce que RLS cache les arêtes mais ne les sort pas du graph.
# print(f"Number of edges in datasets : \n  Hole Dataset : {iric_pyg.num_edges}\n\n  Train: {train_data.num_edges}\n\n  Test : {test_data.num_edges}\n\n  Validation : {val_data.num_edges}")
print(f"Number of edges in datasets : \n  Train: {list(train_data.edge_label.size())[0]}\n\n  Test : {list(test_data.edge_label.size())[0]}\n\n  Validation : {list(val_data.edge_label.size())[0]}")

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

print('\n\n',train_data.num_nodes)
print(train_data.num_edge_types)
print(train_data.__dict__)
print(train_data.edge_index[0].size())
print(train_data.edge_index[1].size())
print(train_data.edge_attr.size())
print(train_data.edge_attr)
print(train_data.num_nodes)
print(train_data.edge_index.size()[1])

Hole Dataset :
 Data(edge_index=[2, 169248], edge_attr=[169248], num_nodes=30396)

Train:
Data(edge_index=[2, 135400], edge_attr=[135400], num_nodes=30396, edge_label=[135400], edge_label_index=[2, 135400])

Test :
Data(edge_index=[2, 152324], edge_attr=[152324], num_nodes=30396, edge_label=[33848], edge_label_index=[2, 33848])

Validation :
Data(edge_index=[2, 135400], edge_attr=[135400], num_nodes=30396, edge_label=[33848], edge_label_index=[2, 33848])
Number of edges in datasets : 
  Train: 135400

  Test : 33848

  Validation : 33848


 30396
1
{'_edge_attr_cls': <class 'torch_geometric.data.data.DataEdgeAttr'>, '_tensor_attr_cls': <class 'torch_geometric.data.data.DataTensorAttr'>, '_store': {'edge_index': tensor([[27102,  4862, 22900,  ...,  6876, 15366, 18701],
        [18656,  8465,  8731,  ..., 29788, 20668, 18000]]), 'edge_attr': tensor([0, 0, 0,  ..., 0, 0, 0]), 'num_nodes': 30396, 'edge_label': tensor([1., 1., 1.,  ..., 1., 1., 1.]), 'edge_label_index': tensor([[27102,  486

## Iniating normal model

In [16]:
complex_model = ComplEx(
    num_nodes=train_data.num_nodes,
    # num_relations=train_data.num_edge_types,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
).to(device)

head_index = train_data.edge_index[0]
tail_index = train_data.edge_index[1]
rel_type = train_data.edge_attr

loader = complex_model.loader(
    head_index = head_index,
    tail_index = tail_index,
    rel_type = rel_type,
    batch_size=batch_size,
    shuffle=True,
    # neg_sampling = 4
)

optimizer = optim.Adam(complex_model.parameters(),
                      #  lr=0.01, weight_decay=1e-6
                       )

## Training ComplEx

In [ ]:
@torch.no_grad()
def test(data):
    complex_model.eval()
    return complex_model.test(
        head_index=test_data.edge_index[0],
        tail_index=test_data.edge_index[1],
        rel_type=test_data.edge_attr,
        batch_size=1000,
        k=10,
    )

def train():
    complex_model.train()
    total_loss = total_examples = 0

    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = complex_model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples

torch.set_grad_enabled(True)
complex_model.to(device)

complex_model.reset_parameters()


start_rank, start_mrr, start_hits_at_10 = test(test_data)
print(f'Start Test Mean Rank: {start_rank:.2f}, Start Test MRR: {start_mrr:.4f}, '
      f'Start Test Hits@10: {start_hits_at_10:.4f}')

losses = []
for epoch in range(1, epochs):
    loss = train()
    losses.append(loss)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 500 == 0:
        rank, mrr, hits = test(val_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}',
              f'Val MRR: {mrr:.4f}, Val Hits@10: {hits:.4f}')

rank, mrr, hits_at_10 = test(test_data)

print(f'Start Test Mean Rank: {start_rank:.2f}, Start Test MRR: {start_mrr:.4f}, '
      f'Start Test Hits@10: {start_hits_at_10:.4f}')
print(f'Final Test Mean Rank: {rank:.2f}, Final Test MRR: {mrr:.4f}, '
      f'Final Test Hits@10: {hits_at_10:.4f}')

torch.set_grad_enabled(False)

 16%|█▋        | 24968/152324 [05:10<23:37, 89.83it/s]

## Testing ComplEx

In [ ]:
def plot_loss(loss_list):
    plt.plot(loss_list, label='Loss')
    plt.title('Evolution des Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def plot_loss_log(loss_list):
    plt.semilogy(loss_list, label='Loss')
    plt.title('Evolution des Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def running_mean(list,
                 half_window: int # Number of elements that the function will consider
                                  # ahead and behind the pos X to calculate running mean at X.
                 ):

    running_means = []

    for i in range(0,len(list)):

        left_bound = max(0,i-half_window)
        right_bound = min(len(list)-1, i + half_window)
        sublist = list[left_bound:right_bound+1]
        running_means.append(sum(sublist)/len(sublist))

    return running_means

plot_loss(losses)
plot_loss_log(losses)
plot_loss_log(running_mean(losses, 30))

In [ ]:
complex_model.to(device)
torch.set_grad_enabled(False)

# Is complex_model coherent ?
iric_first_line = iric.iloc[0].tolist()
map_tail = entity_to_mapping[iric_first_line[0]]
map_head = entity_to_mapping[iric_first_line[2]]
map_rel = relation_to_mapping[iric_first_line[1]]

print("Good score should appear here :")
print(complex_model(torch.tensor(map_tail),torch.tensor(map_rel),torch.tensor(map_head)))



print("Bad scores should appear here :")
print(complex_model(torch.tensor(random.choice(list(mapping_to_entity.keys()))),torch.tensor(0),torch.tensor(random.choice(list(mapping_to_entity.keys())))))
print(complex_model(torch.tensor(random.choice(list(mapping_to_entity.keys()))),torch.tensor(0),torch.tensor(random.choice(list(mapping_to_entity.keys())))))

In [ ]:
model_save_path = "TestComplEx"
loc_path = %pwd
torch.save(complex_model.state_dict(), model_save_path)
print(f'Model saved at {loc_path}/{model_save_path}')

# Iniating model with Lin loss

In [ ]:
nxo = from_file(url)
nxo.freeze()

In [ ]:
possibles_tails: dict = {}
for index, row in iric.iterrows():
    head, rel, tail = row.iloc[0], row.iloc[1], row.iloc[2]
    hr = (head, rel)
    if hr not in possibles_tails:
        possibles_tails[hr]=[tail]
    else :
        possibles_tails[hr].append(tail)

print(possibles_tails)

In [ ]:
def lin_similarity(term1:str, term2:str, ontology, error_return = 0):
  '''
  Returns the highier lin similarity of two entities of a list of given ontologies.
  /!\ If calculation is impossible (for example if one of the entities is in none of the ontologies), return error_return (default 0).

  Parameters :
  - term1 (str): ID of the first entity as a string (example : "GO:0042552")
  - term2 (str): ID of the second entity as a string (example : "GO:0042552")
  - ontology (nxontology.ontology.NXOntology objects) : An ontology loaded with nxontology.
  - error_return : The output of the function if lin similarity cannot be calculated. Default 0.

  Returns :
  - linsim (float): The Lin similarity of the two entities in the ontology.
                    It is a a number in [0;1].
                    linsim(A,B) close of 0 means A and B are distant in the ontology ;
                    lisnsim(A,B) close of 1 means A and B are close entities in the ontology.
  '''
  try :
      return ontology.similarity(term1, term2).lin
  except :
      return error_return

In [ ]:
def LinSim_errors_for_triple(triple: tuple,
                             ontology,
                             possibles_tails_for_head_rel_double:dict,
                             error_return = 0
                                              )-> list :
  '''
  Given a triple (head, relation, tail) and an ontology, calculates the list of LinSim between the real head of the triple and the expected heads for the (head, relation) couple.

  Parameters :
  - triple (tuple): The concatenation of the embeddings of head, relation and tail.
  - ontology (list of nxontology.ontology.NXOntology objects) : An ontology loaded with nxontology.
  - possibles_tails_for_head_rel_double (dict): Associates the list of possible tails

  - error_return : The output of the function if lin similarity cannot be calculated. Default 0.

  Returns :
  - similarities (list) : the list of LinSim between the actual triple head and the possible heads of the triple found in iric.
  '''
  head, relation, tail = triple[0], triple[1], triple[2]

  possibles_tails = possibles_tails_for_head_rel_double[(head, relation)]
  similarities = [lin_similarity(tail, possible_tail, ontology=ontology, error_return=error_return) for possible_tail in possibles_tails]

  return similarities

In [ ]:
class Lin_ComplEx(ComplEx):
  def loss(
        self,
        head_index: torch.Tensor,
        rel_type: torch.Tensor,
        tail_index: torch.Tensor,
    ) -> torch.Tensor:

        pos_score = self(head_index, rel_type, tail_index)
        neg_score = self(*self.random_sample(head_index, rel_type, tail_index))
        scores = torch.cat([pos_score, neg_score], dim=0)

        pos_target = torch.ones_like(pos_score)
        neg_target = torch.zeros_like(neg_score)
        target = torch.cat([pos_target, neg_target], dim=0)

        tail_map =

        return F.binary_cross_entropy_with_logits(scores, target)-
